### PROCESS DATA

In [1]:
import pandas as pd
import numpy as np

In [2]:
def getStateNew(data, t, n):
    d = t - n + 1
    index = data.index.unique()
    block = data.loc[index[d:t + 1]]
    return np.array(block).reshape(1,-1)

# def getStockDataVecNew():
#     df=pd.read_csv("./data/full_data.csv")
#     df = df[['open_time', 'close', 'volume','count', 'taker_buy_volume','symbol']]
#     df.set_index('open_time', inplace=True)
#     df['symbol'][df['symbol']=='ADABUSD'] =0
#     df['symbol'][df['symbol']=='ETHBUSD'] =1
#     df['symbol'][df['symbol']=='BNBBUSD'] =2
#     df['symbol'][df['symbol']=='BTCBUSD'] =3
#     df['symbol'][df['symbol']=='DOTBUSD'] =4
#     return df


In [3]:
def getStockDataVecNew():
    df = pd.read_csv("./data/full_data.csv")
    df = df[['open_time', 'close', 'volume', 'count', 'taker_buy_volume', 'symbol']]
    df.set_index('open_time', inplace=True)
    symbol_mapping = {'ADABUSD': 0, 'ETHBUSD': 1, 'BNBBUSD': 2, 'BTCBUSD': 3, 'DOTBUSD': 4}
    df['symbol'] = df['symbol'].map(symbol_mapping).fillna(df['symbol'])
    return df

# def getStateNew(data, t, n):
#     d = t - n + 1
#     block = data.iloc[d : t + 1].values
#     return block.reshape(1, -1)

In [4]:
index = data.index.unique()
data.loc[index[0:2]]

NameError: name 'data' is not defined

In [7]:
data = getStockDataVecNew()
# def getStateNew(data, t, n):
#     d = t - n + 1
#     index = data.index.unique()
#     block = data.loc[index[d:t + 1]]
#     return block

### Train

In [11]:
from rl_agent.Agent import Agent
from rl_agent.memory import Transition, ReplayMemory
from function import *
import sys
import torch
from param import *

In [19]:
stock_name, window_size, episode_count =  "full_data", 10, 100


agent = Agent(250)
data = getStockDataVec()
l = len(data)
l

99360

In [20]:
l/5

19872.0

In [21]:
19871*5

99355

In [8]:
# next_state = getStateNew(data, t + 1, window_size)
# reward = 0
# next_state.shape

In [ ]:
# next_state = getState(data, t + 5, window_size)

In [38]:
print("Episode " + str(1) + "/" + str(episode_count))
state = getState(data, window_size-1, window_size)

total_profit = 0
agent.inventory = []
print(l)
# round(l/5)-1 -10000
for i in range(1,1147):
    
    t= i*5-1

    # print(t)s
    action = agent.act(state)

    # sit
    next_state = getState(data, t + 5, window_size)
    reward = 0

    if action == 1: # buy
        agent.inventory.append(data.loc[data.index[t]])
        # print("Buy: " + formatPrice(data[t]))

    elif action == 2 and len(agent.inventory) > 0: # sell
        bought_price = agent.inventory.pop(0)
        
        close_price = data[data['symbol']==BTCBUSD].loc[data.index[t]].close
        profit = close_price - bought_price[bought_price['symbol']==BTCBUSD].close.values.item()
        reward = max(profit, 0)
        total_profit += profit
        # print("Sell: " + formatPrice(data[t]) + " | Profit: " + formatPrice(data[t] - bought_price))
        print('total_profit: ',total_profit)

    done = True if t == l - 1 else False
    agent.memory.push(state, action, next_state, reward)
    state = next_state

    if done:
        print("--------------------------------")
        print("Total Profit: " + formatPrice(total_profit))
        print("--------------------------------")

    agent.optimize()

Episode 1/100
99360
total_profit:  29.67000000000189
total_profit:  12.680000000000291
total_profit:  8.479999999999563
total_profit:  -1.3699999999989814
total_profit:  2.349999999998545
total_profit:  26.909999999999854
total_profit:  37.44999999999709
total_profit:  36.54999999999927
total_profit:  60.719999999997526
total_profit:  66.57999999999811
total_profit:  56.219999999997526
total_profit:  67.31999999999607
total_profit:  95.07999999999447
total_profit:  109.89999999999418
total_profit:  107.04999999999563
total_profit:  69.15999999999622
total_profit:  30.2699999999968
total_profit:  -19.270000000004075
total_profit:  -89.06000000000495
total_profit:  -98.09000000000378
total_profit:  -102.33000000000538
total_profit:  -130.2700000000077
total_profit:  -103.9700000000048
total_profit:  -92.81000000000495
total_profit:  -104.37000000000262
total_profit:  -324.32000000000335
total_profit:  -546.7700000000041
total_profit:  -750.6500000000051
total_profit:  -948.6700000000055


250

### old code

In [ ]:
import numpy as np
import math
import pandas as pd

# prints formatted price
def formatPrice(n):
	return ("-$" if n < 0 else "$") + "{0:.2f}".format(abs(n))

# returns the vector containing stock data from a fixed file
# def getStockDataVec():
#     df=pd.read_csv("./data/full_data.csv")
#     df = df[['open_time', 'close', 'volume','count', 'taker_buy_volume','symbol']]
#     df.set_index('open_time', inplace=True)
#     df['symbol'][df['symbol']=='ADABUSD'] =0
#     df['symbol'][df['symbol']=='ETHBUSD'] =1
#     df['symbol'][df['symbol']=='BNBBUSD'] =2
#     df['symbol'][df['symbol']=='BTCBUSD'] =3
#     df['symbol'][df['symbol']=='DOTBUSD'] =4
#     return df

def getStockDataVec():
    df = pd.read_csv("./data/full_data.csv")
    df = df[['open_time', 'close', 'volume', 'count', 'taker_buy_volume', 'symbol']]
    df.set_index('open_time', inplace=True)
    symbol_mapping = {'ADABUSD': 0, 'ETHBUSD': 1, 'BNBBUSD': 2, 'BTCBUSD': 3, 'DOTBUSD': 4}
    df['symbol'] = df['symbol'].map(symbol_mapping).fillna(df['symbol'])
    return df

def getState(data, t, n):
    d = t - n + 1
    index = data.index.unique()
    block = data.loc[index[d:t + 1]]
    return np.array(block).reshape(1,-1)

In [ ]:
from rl_agent.Agent import Agent
from rl_agent.memory import Transition, ReplayMemory
from function import *
from param import *
import sys
import torch


stock_name, window_size, episode_count =  "full_data", 10, 10


agent = Agent(window_size*5*5) # 10 sample and each sample is 5x5 
data = getStockDataVec()
l = len(data) - 1

for e in range(episode_count + 1):
	print("Episode " + str(e) + "/" + str(episode_count))
	state = getState(data, window_size-1, window_size)

	total_profit = 0
	agent.inventory = []
	print(l)
 	# for t in range(window_size-1,l):

	for t in range(window_size-1,1872):
		# print(t)
		action = agent.act(state)

		# sit
		next_state = getState(data, t + 1, window_size)
		reward = 0

		if action == 1: # buy
			agent.inventory.append(data.loc[data.index[t]])
			# print("Buy: " + formatPrice(data[t]))

		elif action == 2 and len(agent.inventory) > 0: # sell
			bought_price = agent.inventory.pop(0)
			
			close_price = data[data['symbol']==BTCBUSD].loc[data.index[t]].close
			profit = close_price - bought_price[bought_price['symbol']==BTCBUSD].close.values.item()
			reward = max(profit, 0)
			total_profit += profit
			# print("Sell: " + formatPrice(data[t]) + " | Profit: " + formatPrice(data[t] - bought_price))
			print('total_profit: ',total_profit)

		done = True if t == l - 1 else False
		agent.memory.push(state, action, next_state, reward)
		state = next_state

		if done:
			print("--------------------------------")
			print("Total Profit: " + formatPrice(total_profit))
			print("--------------------------------")

		agent.optimize()
	if e % 2 == 0:
		agent.target_net.load_state_dict(agent.policy_net.state_dict())
		torch.save(agent.policy_net, "models/policy_model")
		torch.save(agent.target_net, "models/target_model")